# Account Health Scoring Description

Account health scoring predicts how **valuable/engaged/at-risk** a customer account is on a scale of 0-100. It's like a "wellness score" for customer relationships:
- High score (70-100): Healthy, engaged, likely to renew/expand
- Medium score (40-69): Stable but needs attention
- Low score (0-39): At-risk, potential churn

## Step 1: Import Packages

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV

## Step 2: Load and Prepare Data

In [3]:
base_path = os.path.abspath("../data_directory/clean_data")

# Read the data
pipeline = pd.read_csv(os.path.join(base_path, "Pipeline.csv"))
accounts = pd.read_csv(os.path.join(base_path, "Accounts.csv"))
teams = pd.read_csv(os.path.join(base_path, "Teams.csv"))
products = pd.read_csv(os.path.join(base_path, "Products.csv"))

In [4]:
# Understanding each dataset
print("ACCOUNTS:", accounts.shape)
print(accounts.columns.tolist())
print(accounts.head(3))
print("\nPIPELINE:", pipeline.shape)
print(pipeline.columns.tolist())
print(pipeline.head(3))

ACCOUNTS: (85, 7)
['account', 'sector', 'year_established', 'revenue', 'employees', 'office_location', 'subsidiary_of']
            account      sector  year_established  revenue  employees  \
0  acme_corporation  technology            1996.0  1100.04     2822.0   
1        betasoloin     medical            1999.0   251.41      495.0   
2          betatech     medical            1986.0   647.18     1185.0   

  office_location     subsidiary_of  
0   united_states  acme_corporation  
1   united_states  acme_corporation  
2           kenya  acme_corporation  

PIPELINE: (8800, 8)
['opportunity_id', 'sales_agent', 'product', 'account', 'deal_stage', 'engage_date', 'close_date', 'close_value']
  opportunity_id      sales_agent         product  account deal_stage  \
0       1c1i7a6r      moses_frase  gtx_plus_basic  cancity        won   
1       z063oyw0  darcel_schlecht          gtxpro    isdom        won   
2       ec4qe1bx  darcel_schlecht      mg_special  cancity        won   

  engag

## Step 3: Create engagement metrics from Pipeline data

In [5]:
# Merge pipeline with products to get revenue info
pipeline_enhanced = pipeline.merge(products, on='product', how='left')

# Calculate per-account metrics
account_metrics = pipeline.groupby('account').agg({
    'opportunity_id': 'count',  # Total opportunities
    'close_value': ['sum', 'mean', 'std', 'max'],  # Revenue metrics + max deal
    'deal_stage': [
        lambda x: (x == 'Won').sum(),      # Win count
        lambda x: (x == 'Lost').sum(),     # Loss count
        'count'                             # Total deals
    ],
    'engage_date': lambda x: (pd.to_datetime(x).max() - pd.to_datetime(x).min()).days,  # Customer lifetime
    'product': 'nunique',       # Product diversity
    'sales_agent': 'nunique'    # Agent diversity
}).reset_index()

In [6]:
# Flatten column names
account_metrics.columns = [
    'account', 
    'total_opportunities', 
    'total_revenue', 
    'avg_deal_size', 
    'revenue_std', 
    'max_deal_size',           # NEW
    'total_wins', 
    'total_losses',            # NEW
    'total_deals',             # NEW
    'customer_lifetime_days',  # NEW
    'product_diversity', 
    'agent_count'
]

In [7]:
# Calculate the new metrics
account_metrics['win_rate'] = (account_metrics['total_wins'] /  account_metrics['total_opportunities'] * 100)

account_metrics['loss_rate'] = (account_metrics['total_losses'] /  account_metrics['total_opportunities'] * 100)  # NEW

account_metrics['deals_per_year'] = (account_metrics['total_deals'] / (account_metrics['customer_lifetime_days'] / 365 + 0.01))  # NEW (can't divide by zero)

# Merge with accounts data
merged_df = accounts.merge(account_metrics, on='account', how='left')

In [8]:
# Days since last deal
last_deal_dates = pipeline.groupby('account')['close_date'].max()
merged_df['days_since_last_deal'] = (pd.Timestamp.now() - pd.to_datetime(last_deal_dates)).dt.days

In [9]:
# Revenue momentum (recent performance vs historical)
recent_cutoff = '2016-01-01'  # Guesstimate
recent_revenue = (pipeline[pd.to_datetime(pipeline['close_date']) > recent_cutoff]
                  .groupby('account')['close_value'].sum())
merged_df = merged_df.merge(recent_revenue.rename('recent_revenue'), 
                            on='account', how='left')
merged_df['recent_revenue'].fillna(0, inplace=True)
merged_df['revenue_momentum'] = merged_df['recent_revenue'] / (merged_df['total_revenue'] + 1)  # +1 to avoid division by zero


/var/folders/nl/6y23jbvx6976ljxjh0vqgr7w0000gn/T/ipykernel_71888/1984320299.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['recent_revenue'].fillna(0, inplace=True)


In [10]:
print("Merged dataframe shape:", merged_df.shape)
print(f"New features added: {merged_df.shape[1]} total columns")
print("\nColumn names:")
print(merged_df.columns.tolist())
print("\nFirst few rows:")
print(merged_df.head())

Merged dataframe shape: (85, 24)
New features added: 24 total columns

Column names:
['account', 'sector', 'year_established', 'revenue', 'employees', 'office_location', 'subsidiary_of', 'total_opportunities', 'total_revenue', 'avg_deal_size', 'revenue_std', 'max_deal_size', 'total_wins', 'total_losses', 'total_deals', 'customer_lifetime_days', 'product_diversity', 'agent_count', 'win_rate', 'loss_rate', 'deals_per_year', 'days_since_last_deal', 'recent_revenue', 'revenue_momentum']

First few rows:
            account      sector  year_established  revenue  employees  \
0  acme_corporation  technology            1996.0  1100.04     2822.0   
1        betasoloin     medical            1999.0   251.41      495.0   
2          betatech     medical            1986.0   647.18     1185.0   
3        bioholding     medical            2012.0   587.34     1356.0   
4           bioplex     medical            1991.0   326.82     1016.0   

  office_location     subsidiary_of  total_opportunities

In [11]:
print("\nMissing values in new features:")
print(merged_df[['max_deal_size', 'total_losses', 'customer_lifetime_days', 
                  'loss_rate', 'deals_per_year']].isnull().sum())


Missing values in new features:
max_deal_size             0
total_losses              0
customer_lifetime_days    0
loss_rate                 0
deals_per_year            0
dtype: int64


## Step 4: Target Variable

In [12]:
# Used to normalize everything to 0-100 scale
from sklearn.preprocessing import MinMaxScaler

In [13]:
scaler = MinMaxScaler(feature_range=(0, 100))

# Components of health score
components = pd.DataFrame()
components['revenue_score'] = scaler.fit_transform(merged_df[['total_revenue']])[:, 0]
components['win_rate_score'] = merged_df['win_rate']  # Already 0-100
components['engagement_score'] = scaler.fit_transform(merged_df[['total_opportunities']])
components['avg_deal_score'] = scaler.fit_transform(merged_df[['avg_deal_size']])

# Weighted combination (ask JP and Fanizza if this makes sense)
weights = {
    'revenue_score': 0.5,      # 50% (most important)
    'win_rate_score': 0.25,     # 25% (conversion matters)
    'engagement_score': 0.15,   # 15% (activity level)
    'avg_deal_score': 0.1      # 10% (deal quality)
}

y_target = (
    components['revenue_score'] * weights['revenue_score'] +
    components['win_rate_score'] * weights['win_rate_score'] +
    components['engagement_score'] * weights['engagement_score'] +
    components['avg_deal_score'] * weights['avg_deal_score']
)

# Verify it's 0-100
print(f"Target range: {y_target.min():.2f} to {y_target.max():.2f}")
print(f"Target mean: {y_target.mean():.2f}")
print(f"Target distribution:\n{y_target.describe()}")

Target range: 2.19 to 65.00
Target mean: 10.74
Target distribution:
count    85.000000
mean     10.739276
std       7.260205
min       2.192426
25%       7.633275
50%       9.157848
75%      12.071152
max      65.000000
dtype: float64


## Preparing the Features

In [14]:
# Encode categorical variables
merged_df['has_subsidiary'] = (merged_df['subsidiary_of'] != 'Independent').astype(int)

# One-hot encode sector and location
merged_df = pd.get_dummies(merged_df, columns=['sector', 'office_location'], drop_first=True)

# Select features for model
feature_cols = ['revenue', 'employees', 'year_established', 'has_subsidiary',
                'product_diversity', 'agent_count'] + \
               [col for col in merged_df.columns if col.startswith('sector_') or 
                col.startswith('office_location_')]

X = merged_df[feature_cols]
y = y_target

print(f"Features shape: {X.shape}")
print(f"Feature columns: {X.columns.tolist()}")

Features shape: (85, 29)
Feature columns: ['revenue', 'employees', 'year_established', 'has_subsidiary', 'product_diversity', 'agent_count', 'sector_entertainment', 'sector_finance', 'sector_marketing', 'sector_medical', 'sector_retail', 'sector_services', 'sector_software', 'sector_technology', 'sector_telecommunications', 'office_location_brazil', 'office_location_china', 'office_location_germany', 'office_location_italy', 'office_location_japan', 'office_location_jordan', 'office_location_kenya', 'office_location_korea', 'office_location_norway', 'office_location_panama', 'office_location_philippines', 'office_location_poland', 'office_location_romania', 'office_location_united_states']


## Train Models

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Testing only the top 3 scaling methods to save time
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0), # From JP's notes
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
}

scaling_methods = ['normalize', 'clipping', 'direct']  # Starting w/ just with these 3

results = {}

for model_name, model in models.items():
    for scaling in scaling_methods:
        # Train model
        if scaling == 'normalize':
            y_train_scaled = y_train / 100
            model.fit(X_train_scaled, y_train_scaled)
            preds = model.predict(X_test_scaled) * 100
        else:
            model.fit(X_train_scaled, y_train)
            preds = model.predict(X_test_scaled)
        
        # Apply scaling
        if scaling == 'clipping':
            preds = np.clip(preds, 0, 100)
        
        # Calculate metrics
        r2 = r2_score(y_test, preds) # Measures overall model fit
        mae = mean_absolute_error(y_test, preds) # More of a general error measurement
        rmse = np.sqrt(mean_squared_error(y_test, preds)) # Penalizes large errors more than MAE
        
        results[f"{model_name}_{scaling}"] = {
            'R2': round(r2, 4),
            'MAE': round(mae, 2),
            'RMSE': round(rmse, 2)
        }

# Show results
results_df = pd.DataFrame(results).T
print("\nMODEL RESULTS")
print(results_df.sort_values('R2', ascending=False))


MODEL RESULTS
                                 R2   MAE   RMSE
Linear Regression_normalize  0.4064  4.60  10.28
Linear Regression_clipping   0.4064  4.60  10.28
Linear Regression_direct     0.4064  4.60  10.28
Ridge_normalize              0.3908  4.51  10.42
Ridge_clipping               0.3908  4.51  10.42
Ridge_direct                 0.3908  4.51  10.42
Random Forest_normalize      0.2973  4.55  11.19
Random Forest_clipping       0.2862  4.58  11.28
Random Forest_direct         0.2862  4.58  11.28


## Evaluation

After evaluating, the best model is linear regression with any scaling method! It has the **highest R2 score** (explains the most variance) and the **lowest RMSE** (best at avoiding large prediction errors). It's also the **simplest model** and scaling didn't matter because all three scaling methods give identical results.

**However, the R2 score is not > 0.7 as is required which means the model is only 38% accurate at predicting account health.**

Ridge performed slightly worse than linear regression which means there's no significant overfitting in the data (yay!).

Random forest was the worst which I think is because the dataset is too small for that model.